## Importing PWE and other essentials

In [1]:
from PW_explorer.run_clingo import run_clingo
from PW_explorer.load_worlds import load_worlds
from PW_explorer.pwe_query import PWEQuery
from PW_explorer.pwe_helper import pw_slicer, rel_slicer
from PW_explorer.time_series import PWETimeSeriesModule
import pandas as pd
import numpy as np
import PW_explorer as pwe

## Formulating the Towers of Hanoi Problem

This is the towers of Hanoi instance

In [2]:
with open('clingo_files/toh_ins.lp') as f:
    toh_instance = list(map(str.strip, f.readlines()))
print("\n".join(toh_instance))

#const num_disks=3.
% define peg(PEG)
peg(a;b;c).
% define disk(DISK)
disk(1..num_disks).
% define init_on(DISK, PEG)
init_on(1..num_disks,a).
% define goal_on(DISK, PEG)
goal_on(1..num_disks,c).
moves(7).


This is the encoding of the rules of the game

In [3]:
with open('clingo_files/toh_enc.lp') as f:
    toh_encoding = list(map(str.strip, f.readlines()))
print("\n".join(toh_encoding))

% Generate
{ move(D,P,T) : disk(D), peg(P) } = 1 :- moves(M),  T = 1..M.
% define move(DISK, PEG, TIME)
% temporal move(_,T)
% define move(DISK, TIME)
% temporal move(_,_,T)
% define on(DISK, PEG, TIME)
% temporal on(_,_,T)
% define blocked(DISK, PEG, TIME)
% temporal blocked(_,_,T)
move(D,T)   :- move(D,_,T).
on(D,P,0)   :- init_on(D,P).
on(D,P,T)   :- move(D,P,T).
on(D,P,T+1) :- on(D,P,T), not move(D,T+1), not moves(T).
blocked(D-1,P,T+1) :- on(D,P,T), not moves(T).
blocked(D-1,P,T)   :- blocked(D,P,T), disk(D).
% Test
:- move(D,P,T), blocked(D-1,P,T).
:- move(D,T), on(D,P,T-1), blocked(D,P,T).
:- goal_on(D,P), not on(D,P,M), moves(M).
:- { on(D,P,T) } != 1, disk(D), moves(M), T = 1..M.
%% % Display
%% #show move/3.


In [4]:
clingo_rules = toh_encoding+toh_instance

Running this carefully constructed to yield exactly one PW example with clingo 

In [5]:
clingo_soln, meta_data = run_clingo(clingo_rules=clingo_rules)
print('\n'.join(clingo_soln))

Answer: 1
disk(1) disk(2) disk(3) peg(a) peg(b) peg(c) moves(7) init_on(1,a) init_on(2,a) init_on(3,a) on(1,a,0) on(2,a,0) on(3,a,0) blocked(0,a,1) blocked(1,a,1) blocked(2,a,1) goal_on(1,c) goal_on(2,c) goal_on(3,c) on(1,a,1) on(2,a,1) blocked(0,a,2) blocked(1,a,2) move(3,1) move(3,c,1) move(2,2) move(2,b,2) move(3,3) move(3,b,3) move(1,4) move(1,c,4) move(3,5) move(3,a,5) move(2,6) move(2,c,6) move(3,7) move(3,c,7) on(3,c,1) on(1,a,2) on(2,b,2) on(3,c,2) on(1,a,3) on(2,b,3) on(3,b,3) on(1,c,4) on(2,b,4) on(3,b,4) on(1,c,5) on(2,b,5) on(3,a,5) on(1,c,6) on(2,c,6) on(3,a,6) on(1,c,7) on(2,c,7) on(3,c,7) blocked(0,c,2) blocked(1,c,2) blocked(2,c,2) blocked(0,a,3) blocked(0,b,3) blocked(0,c,3) blocked(1,b,3) blocked(1,c,3) blocked(2,c,3) blocked(0,a,4) blocked(0,b,4) blocked(1,b,4) blocked(2,b,4) blocked(0,b,5) blocked(0,c,5) blocked(1,b,5) blocked(2,b,5) blocked(0,a,6) blocked(0,b,6) blocked(0,c,6) blocked(1,a,6) blocked(1,b,6) blocked(2,a,6) blocked(0,a,7) blocked(0,c,7) blocked(1,a,7)

This is the extracted meta data for the ASP formulation

In [6]:
meta_data

{'attr_defs': {'move_3': ['DISK', 'PEG', 'TIME'],
  'move_2': ['DISK', 'TIME'],
  'on_3': ['DISK', 'PEG', 'TIME'],
  'blocked_3': ['DISK', 'PEG', 'TIME'],
  'peg_1': ['PEG'],
  'disk_1': ['DISK'],
  'init_on_2': ['DISK', 'PEG'],
  'goal_on_2': ['DISK', 'PEG']},
 'temporal_decs': {'move_2': [1],
  'move_3': [2],
  'on_3': [2],
  'blocked_3': [2]}}

Now we load the solution using PWE

In [7]:
pw_rels_dfs, rel_schemas, pw_objs = load_worlds(asp_output=clingo_soln, meta_data=meta_data, reasoner='clingo')

Number of Models: 1


In [8]:
list(map(lambda x: x.__dict__, rel_schemas))

[{'relation_name': 'disk_1',
  'arity': 1,
  'r_id': 0,
  'meta_data': {'attr_defs': ['DISK']}},
 {'relation_name': 'peg_1',
  'arity': 1,
  'r_id': 1,
  'meta_data': {'attr_defs': ['PEG']}},
 {'relation_name': 'moves_1', 'arity': 1, 'r_id': 2, 'meta_data': {}},
 {'relation_name': 'init_on_2',
  'arity': 2,
  'r_id': 3,
  'meta_data': {'attr_defs': ['DISK', 'PEG']}},
 {'relation_name': 'on_3',
  'arity': 3,
  'r_id': 4,
  'meta_data': {'attr_defs': ['DISK', 'PEG', 'TIME'], 'temporal_decs': [2]}},
 {'relation_name': 'blocked_3',
  'arity': 3,
  'r_id': 5,
  'meta_data': {'attr_defs': ['DISK', 'PEG', 'TIME'], 'temporal_decs': [2]}},
 {'relation_name': 'goal_on_2',
  'arity': 2,
  'r_id': 6,
  'meta_data': {'attr_defs': ['DISK', 'PEG']}},
 {'relation_name': 'move_2',
  'arity': 2,
  'r_id': 7,
  'meta_data': {'attr_defs': ['DISK', 'TIME'], 'temporal_decs': [1]}},
 {'relation_name': 'move_3',
  'arity': 3,
  'r_id': 8,
  'meta_data': {'attr_defs': ['DISK', 'PEG', 'TIME'], 'temporal_decs': 

We have parsed the following relations

In [9]:
pw_rels_dfs.keys()

dict_keys(['disk_1', 'peg_1', 'moves_1', 'init_on_2', 'on_3', 'blocked_3', 'goal_on_2', 'move_2', 'move_3'])

And they are each stored in a Pandas Dataframe which looks like this:

(Also notice that the column names were inferred from the meta_data we parsed earlier)

In [10]:
pw_rels_dfs['on_3']

,pw,DISK,PEG,TIME
0,1,1,a,0
1,1,2,a,0
2,1,3,a,0
3,1,1,a,1
4,1,2,a,1
5,1,3,c,1
6,1,1,a,2
7,1,2,b,2
8,1,3,c,2
9,1,1,a,3


##### All these are hard to make sense of, especially since there's time states involved.
##### A simple groupby on the temporal columns would go a long way

In [11]:
sliced_dfs, sliced_rels, _ = rel_slicer(pw_rels_dfs, rel_schemas, None, ['move_3', 'on_3', 'init_on_2', 'peg_1', 'goal_on_2', 'disk_1'])
timestep_state_map = PWETimeSeriesModule.group_by_time(sliced_dfs, sliced_rels)

In [12]:
timestep_state_map

{'constant': {'disk_1':    pw DISK
  0   1    1
  1   1    2
  2   1    3, 'goal_on_2':    pw DISK PEG
  0   1    1   c
  1   1    2   c
  2   1    3   c, 'peg_1':    pw PEG
  0   1   a
  1   1   b
  2   1   c, 'init_on_2':    pw DISK PEG
  0   1    1   a
  1   1    2   a
  2   1    3   a}, 0: {'on_3':    pw DISK PEG
  0   1    1   a
  1   1    2   a
  2   1    3   a}, 1: {'on_3':    pw DISK PEG
  3   1    1   a
  4   1    2   a
  5   1    3   c, 'move_3':    pw DISK PEG
  0   1    3   c}, 2: {'on_3':    pw DISK PEG
  6   1    1   a
  7   1    2   b
  8   1    3   c, 'move_3':    pw DISK PEG
  1   1    2   b}, 3: {'on_3':     pw DISK PEG
  9    1    1   a
  10   1    2   b
  11   1    3   b, 'move_3':    pw DISK PEG
  2   1    3   b}, 4: {'on_3':     pw DISK PEG
  12   1    1   c
  13   1    2   b
  14   1    3   b, 'move_3':    pw DISK PEG
  3   1    1   c}, 5: {'on_3':     pw DISK PEG
  15   1    1   c
  16   1    2   b
  17   1    3   a, 'move_3':    pw DISK PEG
  4   1    3   a}, 6

Not very fun to look at either

#### So we can use this built-in text-based visualization for a clearer understanding of the solution

In [13]:
PWETimeSeriesModule.simple_timeseries_text_visualization(timestep_state_map, jupyter=True)

Constants:

disk_1:


,pw,DISK
0,1,1
1,1,2
2,1,3




goal_on_2:


,pw,DISK,PEG
0,1,1,c
1,1,2,c
2,1,3,c




peg_1:


...




---------------
Timestep 7:
---------------

on_3:


,pw,DISK,PEG
21,1,1,c
22,1,2,c
23,1,3,c




move_3:


,pw,DISK,PEG
6,1,3,c



END


### Building a Custom Visualization for the TOH problem

In [14]:
# More informative custom visualization (still a WIP)
# Inspired by: https://stackoverflow.com/questions/49391076/illustrate-tower-of-hanoi-with-ascii
def toh_viz(timestep_state_map):
    
    NUM_DISKS = len(timestep_state_map['constant']['disk_1'])
    NUM_PEGS = len(timestep_state_map['constant']['peg_1'])
    PEG_SIZE = NUM_DISKS + 2
    
    def get_state(state_df):
        pegs = [[] for _ in range(NUM_PEGS)]
        for i, row in state_df.iterrows():
            pegs[ord(row['PEG']) - ord('a')].append(NUM_DISKS+1-int(row['DISK']))
        pegs = [sorted(peg) for peg in pegs]
        return pegs
    
    def render_ring(ring):
        result = '*' * ring  # the character * repeated ring times.
        return result.center(PEG_SIZE) # add the spaces required
    def render_tower(tower, tower_id):
        result = [render_ring(0) for _ in range(NUM_DISKS-len(tower))] #[]
        for ring in tower:
            result.append(render_ring(ring))
        result.append(str(tower_id).center(PEG_SIZE))
        return result
    def render_final(towers): 
        tower_results = []
        for i, tower in enumerate(towers):
            tower_results.append(render_tower(tower, chr(ord('a')+i)))
        result = []
        for all_rows in zip(*tower_results):
            result.append(''.join(all_rows)) 
        return result
    
    print("Initial State:")
    print('\n'.join(render_final(get_state(timestep_state_map['constant']['init_on_2']))))
    print("")
    print("Goal:")
    print('\n'.join(render_final(get_state(timestep_state_map['constant']['goal_on_2']))))
    print("")
    for t in sorted(set(timestep_state_map.keys()) - set(['constant'])):
        print("Timestep: {}".format(t))
        print("")
        if 'move_3' in timestep_state_map[t]:
            for i, row in timestep_state_map[t]['move_3'].iterrows():
                print("Move {} to peg {}.".format('*'*(NUM_DISKS+1-int(row['DISK'])), row['PEG']))
        print('\n'.join(render_final(get_state(timestep_state_map[t]['on_3']))))
        print("")
    

##### Now let's use the above user-defined visualization

In [15]:
toh_viz(timestep_state_map)

Initial State:
  *            
  **           
 ***           
  a    b    c  

Goal:
            *  
            ** 
           *** 
  a    b    c  

Timestep: 0

  *            
  **           
 ***           
  a    b    c  

Timestep: 1

Move * to peg c.
               
  **           
 ***        *  
  a    b    c  

Timestep: 2

Move ** to peg b.
               
               
 ***   **   *  
  a    b    c  

Timestep: 3

Move * to peg b.
               
       *       
 ***   **      
  a    b    c  

Timestep: 4

Move *** to peg c.
               
       *       
       **  *** 
  a    b    c  

Timestep: 5

Move * to peg a.
               
               
  *    **  *** 
  a    b    c  

Timestep: 6

Move ** to peg c.
               
            ** 
  *        *** 
  a    b    c  

Timestep: 7

Move * to peg c.
            *  
            ** 
           *** 
  a    b    c  

